# LeNet-5 구현(이미지)
## 구현 설계
- 라이브러리 선정
- 데이터를 불러와야 됨
- 간단한 CNN 모델을 작성
- 학습 및 평가

In [1]:
import random
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn 
import torch.optim as optim

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

from PIL import Image

plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

random.seed(42)

cpu


In [2]:
class ImageTransform:
    def __init__(self, resize, mean, std):
        self.data_transform = {
            "train": transforms.Compose([
                transforms.Resize((resize, resize)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
                ]),
            "val": transforms.Compose([
                transforms.Resize((resize, resize)),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
                ]),
            "test": transforms.Compose([
                transforms.Resize((resize, resize)),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
                ]),
        }
    def __call__(self, img, phase):
        return self.data_transform[phase](img)

In [3]:
class DogVsCatDataset(Dataset):
    def __init__(self, file_list, transform=None, phase="train"):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path).convert("RGB")
        img_transform = self.transform(img, self.phase)
        if "dog" in img_path.lower():
            label = 1
        else:
            label = 0
        return img_transform, label            

In [4]:


cat_directory = Path("data/catanddog/train/Cat")
dog_directory = Path("data/catanddog/train/Dog")
cat_images_filepaths = sorted([str(p) for p in cat_directory.glob("*.jpg")])
dog_images_filepaths = sorted([str(p) for p in dog_directory.glob("*.jpg")])
images_filepath = cat_images_filepaths + dog_images_filepaths
random.shuffle(images_filepath)

In [5]:
total_images = len(images_filepath)
train_size = int(total_images * 0.7)
val_size = int(total_images * 0.2)
test_size = total_images - (train_size + val_size)

In [6]:
train_images_filepaths = images_filepath[:train_size]
val_images_filepaths = images_filepath[train_size: train_size + val_size]
test_images_filepaths = images_filepath[train_size + val_size: train_size + val_size + test_size]

In [7]:
size = 224,
mean= (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
batch_size = 32

In [8]:
train_dataset = DogVsCatDataset(train_images_filepaths, transform=ImageTransform(size, mean, std), phase="train")
val_dataset = DogVsCatDataset(val_images_filepaths, transform=ImageTransform(size, mean, std), phase="val")
test_dataset = DogVsCatDataset(test_images_filepaths, transform=ImageTransform(size, mean, std), phase="test")